In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math

from sklearn.model_selection import train_test_split
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data.dataset import Dataset
import warnings

import torch.nn as nn
import torch.nn.functional as F
import scipy.sparse as sp
from sklearn import preprocessing
import dgl
torch.set_printoptions(profile="full")
warnings.filterwarnings('ignore')

Using backend: pytorch


In [3]:
import torch.nn.functional as F
from sklearn.utils import shuffle


df = pd.read_csv('training_5rep_fraclabel.csv')
df.drop("Unnamed: 0", axis = 1, inplace=True)


In [ ]:
def normalize_adj(adj):
    return adj


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def graphencoder(adj):
    source = []
    target = []
    for i in range(adj.shape[0]):
        for j in range(adj.shape[1]):
            if adj[i, j]:
                source.append(i)
                target.append(j)
    source = torch.from_numpy(np.array(source))
    target = torch.from_numpy(np.array(target)) 

    adj = dgl.add_self_loop(dgl.graph((source, target),  num_nodes = 21))
   
    return adj



In [ ]:
class DatasetFromCSV(Dataset):
    def __init__(self, data):
 
        self.data = data.drop("label", axis = 1)
        self.labels = np.asarray(data["label"])
        
        #self.transforms = transforms
        self.train_X, self.test_X, self.train_y, self.test_y = train_test_split(self.data,
                                                   self.labels,
                                                   train_size = 0.8,
                                                   random_state = 1)
        b, s ,t, w= np.unique(self.train_y,return_counts=True,return_index=True,return_inverse=True)
        

 
    def __getitem__(self, index):
        
        single_ec_label = self.train_y[index]
        adj = self.train_X.values[index][0:441].reshape(21, 21)
        features = self.train_X.values[index][441:1638].reshape(21, 57)
        
        
        nor_adj = graphencoder(adj)
        
        #nor_adj = (adj + sp.eye(adj.shape[0]))
        nor_features = normalize(features)
        
        #nor_adj = torch.from_numpy(nor_adj)
        nor_features = torch.from_numpy(nor_features)
        nor_adj.ndata['feats'] = nor_features
    
            
        return (nor_adj, single_ec_label) 
    
    def __len__(self):
        
        return len(self.train_X.index)

In [ ]:
class TestDatasetFromCSV(Dataset):
    def __init__(self, data):
 
        self.data = data.drop("label", axis = 1)
        self.labels = np.asarray(data["label"])
        
        #self.transforms = transforms
        self.train_X, self.test_X, self.train_y, self.test_y = train_test_split(self.data,
                                                   self.labels,
                                                   train_size = 0.8,
                                                   random_state = 1)
        b, s ,t, w= np.unique(self.train_y,return_counts=True,return_index=True,return_inverse=True)
        

 
    def __getitem__(self, index):
        
        single_ec_label = self.test_y[index]
        adj = self.test_X.values[index][0:441].reshape(21, 21)
        features = self.test_X.values[index][441:1638].reshape(21, 57)
    
        nor_adj = graphencoder(adj)
        
        #nor_adj = (adj + sp.eye(adj.shape[0]))
        nor_features = normalize(features)
        
        #nor_adj = torch.from_numpy(nor_adj)
        nor_features = torch.from_numpy(nor_features)
        nor_adj.ndata['feats'] = nor_features
    
            
        return (nor_adj, single_ec_label) 
    
    def __len__(self):
        
        return len(self.test_X.index)

In [ ]:
def collate(samples):
    
    graphs, labels = zip(*samples)
    batched_graph = dgl.batch(graphs)
    
    batched_labels = torch.tensor(labels)
    
    return batched_graph, batched_labels

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(
    DatasetFromCSV(df),
    batch_size=1,
    collate_fn=collate,
    drop_last=False,
    shuffle=True)

In [ ]:
len(DatasetFromCSV(df))

In [ ]:
from torch.utils.data import DataLoader
dataloader_t = DataLoader(
    TestDatasetFromCSV(df),
    batch_size=1,
    collate_fn=collate,
    drop_last=False,
    shuffle=True)

In [ ]:
len(TestDatasetFromCSV(df))

In [ ]:
from dgl.nn.pytorch import edge_softmax, GATConv
class GAT(nn.Module):
    def __init__(self,
                 num_layers,
                 in_dim,
                 num_hidden,
                 num_classes,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop,
                 negative_slope,
                 residual):
        super(GAT, self).__init__()
        self.num_layers = num_layers
        self.gat_layers = nn.ModuleList()
        self.activation = activation
        # input projection (no residual)
        self.gat_layers.append(GATConv(
            in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation))

        # hidden layers
        for l in range(1, num_layers):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.gat_layers.append(GATConv(
                num_hidden * heads[l - 1], num_hidden, heads[l],
                feat_drop, attn_drop, negative_slope, residual, self.activation))
        # output projection
        self.gat_layers.append(GATConv(
            num_hidden * heads[-2], num_classes, heads[-1],
            feat_drop, attn_drop, negative_slope, residual, None))

        #self.bn = nn.BatchNorm1d(num_hidden)
        self.fc = nn.Linear(num_classes, 1)
#         self.do = nn.Dropout(p = 0.5)
#         self.fc2 = nn.Linear(16, 8)
#         self.fc3 = nn.Linear(8, 1)

    def forward(self, g, inputs):
        h = inputs
        for l in range(self.num_layers):
            h = self.gat_layers[l](g, h).flatten(1)
        # output projection
        h = self.gat_layers[-1](g, h).mean(1)
        #h = self.bn(h)
        with g.local_scope():
#             h = F.relu(self.do(self.fc(h)))
#             h = F.relu(self.fc2(h))
#             h = self.fc3(h)
            #h = self.fc2(h)
            g.ndata['h'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, "h")
            # print(hg.size())

            return self.fc(hg)

In [ ]:

heads = ([3] * 3) + [1] 
model = GAT(num_layers = 3,
          in_dim = 57,
          num_hidden = 8,
          num_classes = 6,
          heads = heads,
          activation = F.elu,
          feat_drop = 0。1,
          attn_drop = 0,
          negative_slope = 0,
          residual = False)

model.double()

In [ ]:

opt = torch.optim.Adam(model.parameters(), lr = 0.005)
running_loss = 0
test_loss = 0
total = 0
total_t = 0
epoch_num = 0

for epoch in range(50):

    i = 0
    running_loss = 0.0
    epoch_num += 1
    total = 0
    for batched_graph, labels in dataloader:

        feats = batched_graph.ndata['feats']
        opt.zero_grad()
        model.train()
        outputs = model(batched_graph, feats)
        loss = nn.MSELoss()(outputs, labels)

        running_loss += loss.item()
        total += labels.size(0)  

        loss.backward()
        opt.step()
        i += 1

        if i % 15000 == 14999:
            correct = 0
            total_t = 0
            with torch.no_grad():
                j = 0
                for batched_graph_t, labels_t in dataloader_t:

                    feats_t = batched_graph_t.ndata['feats']
                    model.eval()
                    outputs_t = model(batched_graph_t, feats_t)

                    #loss_t = nn.CrossEntropyLoss()(outputs_t, labels_t)
                    loss_t = nn.MSELoss()(outputs_t, labels_t)
                    test_loss += loss_t.item()

                    #total_t += labels_t.size(0)  
                    #_, predicted = torch.max(outputs_t.data, 1)
                    #correct += (predicted == labels_t).sum().item()
                    #print(predicted, labels_t)
                    j += 1

            print('epoch %d' % epoch_num, 'training_loss: %.4f' % (running_loss / 10000), 'test_loss: %.4f' %(test_loss /j)) #'Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total_t))           
            running_loss = 0.0
            test_loss = 0
            total = 0



In [ ]:
torch.save({
       
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict(),
      
       
        }, "gat_oct18_test.pt")

print("success")